# Using HathiMeta

** Update, SQLite3 seems to have issues, so I replaced it with a dask+parquet backend. **

In [36]:
import dask.dataframe as dd
import pandas as pd

In [1]:
%load_ext autoreload
%autoreload 2
from compare_tools.hathimeta import HathiMeta, get_json_meta
from compare_tools.configuration import config
config.update(config['full'])

## Connecting to the Metadata store

In [37]:
print('Connecting to store at', config['metadb_path'])
meta = HathiMeta(config['metadb_path'])

Connecting to store at /data/saddl/meta.parquet


### First time: Building DB

If you haven't yet ingested the [Hathifiles](https://www.hathitrust.org/hathifiles) or have a transient in-memory db, you'll need to run `create_db`:

In [ ]:
meta.create_db(config['meta_path'])

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

## Access

In [217]:
meta['uiuo.ark:/13960/t6g20tp0c']

access                                                                 allow
rights                                                                    pd
ht_bib_key                                                          11639242
description                                                             v.13
source                                                                   UIU
source_bib_num                                                       2420951
oclc_num                                                            49472649
isbn                                                                    None
issn                                                                    None
lccn                                                                    None
title                      Standard shop efficiency schedules, by Henry W...
imprint                                               Crane & company, 1910-
rights_reason_code                                                       bib

In [218]:
meta.default_fields = ['title', 'author', 'description']
meta['uiuo.ark:/13960/t6g20tp0c']

title          Standard shop efficiency schedules, by Henry W...
author                              Jacobs, Henry William, 1874-
description                                                 v.13
Name: uiuo.ark:/13960/t6g20tp0c, dtype: object

### Get a random item

In [212]:
meta.sample()

title          Agricultural economic report / United States D...
author                                                      None
description                                      no.111-120 1967
Name: osu.32435030095483, dtype: object

In [213]:
meta.sample(2)

,title,author,description
htid,,,
wu.89098276231,A comparison of perceptions of hypertensives a...,"Stright, Patricia A.",None
wu.89085940294,Introductory index of Norwegian handicrafts de...,"Smedal, Elaine Alvine Gudrun, 1922-",None


## Get the full table

This isn't the primary use for HathiMeta, but it may be convenient. Chunk iteration is supported.

In [219]:
meta.get_fields(fields=['title', 'description']).shape

(8741223, 2)

In [230]:
chunks = meta.get_fields(fields=['title', 'description'], by_partition=True)
for i, chunk in enumerate(chunks):
    if i % 10 == 0:
        print(i, chunk.iloc[0]['title'])

0 Memorials of the life of Edward & Lydia Ann Jackson by N. Burwash ; with discourses preached in the Centenary Church, Hamilton, on the occasion of their death / by W.J. Hunter and John Potts.
10 Official Army register for ... / Adjutant General's Office.
20 Message to the Legislature / State of New York.
30 South Tarawa report on the 1990 & 1995 census of population.
40 Worcester sects; or, A history of the Roman Catholics & dissenters of Worcester ...
50 The Yanomami Indian Park : a call for action : a report / by the Anthropology Resource Center.
60 Annual report
70 The functional versus the representational theories of knowledge in Locke's Essay.
80 U.S. Forest Service research note PNW / Pacific Northwest Forest and Range Experiment Station.


## Extending DB

For the SADDL project, we extend the metadata with page_count info, which is available in the Extracted Features data. This is done by passing a basic DataFrame with [htid, newcol1, newcol2] to `extend_db`.

In [ ]:
### if not precomputed, pull from EF data:
#htids = meta.get_fields(['htid'])
#htids['page_count'] = htids.htid.apply(lambda x: get_json_meta(x, config['parquet_root'])['page_count'])
#htids.head(2)
page_counts = pd.read_csv('/data/hathifiles/page_counts.csv', sep='\t', names=['htid', 'page_count']).set_index('htid')

In [10]:
# Where the magic happens
meta.extend_db(page_counts)

In [52]:
meta.get_volume('uiuo.ark:/13960/t6g20tp0c', ['title', 'page_count'])

title         Standard shop efficiency schedules, by Henry W...
page_count                                                  800
Name: uiuo.ark:/13960/t6g20tp0c, dtype: object

Another activity done regularly in running a normalization action on the description field and the title field.

In [3]:
from compare_tools.hathimeta import clean_description, clean_title
to_clean = meta.get_fields(['htid', 'title', 'description'])
# These functions are inconsistently designed, I know - clean_title is an elementwise apply,
# where clean_description uses the vectorized string methods in Pandas, so it takes a column (Series) as input
to_clean['clean_title'] = to_clean.title.fillna('').apply(clean_title)
to_clean['clean_description'] = clean_description(to_clean.description)
meta.extend_db(to_clean[['htid', 'clean_title', 'clean_description']])
to_clean.shape

(8741305, 3)

#### Command-Line Alt
The `apply` in the first step might be inefficient at scale, so there's a script for extracting a metadata field from all of EF which can be parallelized (minimally, because of I/O), and then the page counts can be cross-referenced later. e.g.

`cat all-eng-ef-hathifiles-11-18.txt | parallel --eta -n1000 -j2 python extract_ef_meta.py {} >/data/hathifiles/page_counts.csv`

In [ ]:
import pandas as pd
import numpy as np
pc = pd.read_csv('/data/hathifiles/page_counts.csv', sep='\t', names=['htid', 'page_count'])
pc.page_count = pc.page_count.replace('None',np.nan)
meta.extend_db(pc)